In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import pandas as pd

In [12]:
import hvplot.xarray

In [2]:
# parameters
cloudnet_path = "/home/jovyan/shared_materials/challenge_2_data/cloudnet/"
start_date = "2021-01-01"
end_date = "2024-03-19"

In [3]:
!ls -1 {cloudnet_path} | head -n4
!ls -1 {cloudnet_path} | tail -n4

20210901_mindelo_pollyxt.nc
20210909_mindelo_pollyxt.nc
20210910_mindelo_pollyxt.nc
20210911_mindelo_pollyxt.nc
ls: write error: Broken pipe
LICENSE.txt
README.md
mindelo_pollyxt_daily_means.nc
test.cloudnet.zip


In [4]:
dates = pd.date_range(start=start_date, end=end_date, freq="1D")
files = [sorted(Path(cloudnet_path).glob(f"{d}_mindelo_pollyxt*.nc")) for d in dates.to_series().dt.strftime("%Y%m%d")]
files = list(filter(lambda f: len(f) > 0, files))
files = list(map(lambda f: f[0], files))
files[:10], files[-10:], len(files)

([PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210901_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210909_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210910_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210911_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210912_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210913_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210914_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210915_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210916_mindelo_pollyxt.nc'),
  PosixPath('/home/jovyan/shared_materials/challenge_2_data/cloudnet/20210917_mindelo_pollyxt.nc')],

In [5]:
import tqdm

In [6]:
_daily_means = [xr.open_dataset(fn) for fn in tqdm.tqdm(files[:123])]
_daily_means = [_ds.mean("time").assign_coords(time=_ds.time.mean()) for _ds in tqdm.tqdm(_daily_means)]

100%|██████████| 123/123 [02:12<00:00,  1.08s/it]


In [7]:
daily_means = xr.concat(_daily_means, dim="time")

In [8]:
daily_means.drop_encoding().to_netcdf(
    Path(cloudnet_path) / "mindelo_pollyxt_daily_means.nc"
)

In [13]:
daily_means.sel(range=slice(0, 2000)).depolarisation.T.hvplot()

:Image   [time,range]   (depolarisation)